In [ ]:
import os
import sys
import re
import math
from tqdm import tqdm
from bisect import bisect_right
from termcolor import colored

import matplotlib.patches as mpatch
from scipy import stats
from cycler import cycler

import glob

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from misc.utils import cached, parse_as

# TODO: This is useless for some reason (goal is to make graphs in notebook large and hi-res)
plt.rcParams['figure.dpi'] = 300

%matplotlib inline

# User Settings

In [ ]:
##################################################
# User Settings
##################################################
# Change these to influence the execution of the
# notebook.
# You can override these from the command line
# by defining environment variables with the
# name of the constants below, prefixed with
# "ANA_NB_".
# For example, the environment variable
# "ANA_NB_TR_PATH" will override the "TR_PATH"
# setting below.
##################################################

# The path to the build folder of a ROS2 workspace that contains the
# tracetools_read and tracetools_analysis folders.
TRACING_WS_BUILD_PATH = "~/Projects/autoware/build"

# Path to trace directory (e.g. ~/.ros/my-trace/ust) or to a converted trace file.
# Using the path "/ust" at the end is optional but greatly reduces processing time
# if kernel traces are also present.
# TR_PATH = "/home/max/Downloads/iteration2_worker1/aw_replay/tracing/scenario-trace/ust"
TR_PATH = "data/trace-awsim-x86/ust"

# Path to the folder all artifacts from this notebook are saved to.
# This entails plots as well as data tables.
OUT_PATH = "out/"

# Whether to cache the results of long computations per set of inputs
CACHING_ENABLED = False

# Whether to annotate topics/publications with bandwidth/message size
BW_ENABLED = False
# Path to a results folder as output by ma-hw-perf-tools/messages/record.bash
# Used to annotate message sizes in E2E latency calculations
BW_PATH = "../ma-hw-perf-tools/data/results"

# Whether to use dependencies extracted by the Clang-tools to supplement
# automatic node-internal data flow annotations.
# If in doubt, set to False.
CL_ENABLED = False
# Path to the output directory of the ROS2 dependency checker.
# Will only be used if CL_ENABLED is True.
CL_PATH = "~/Projects/llvm-project/clang-tools-extra/ros2-internal-dependency-checker/output"

# Whether to compute data flow graphs.
# If you are only interested in E2E latencies, set this to False
DFG_ENABLED = True
# Whether to plot data flow graphs (ignored if DFG_ENABLED is False)
DFG_PLOT = True

# The maximum node namespace hierarchy level to be plotted.
# Top-level (1): e.g. /sensing, /control, etc.
# Level 3: e.g. /sensing/lidar/pointcloud_processor
DFG_MAX_HIER_LEVEL = 100

# RegEx pattern for nodes that shall be marked as system inputs
# These will be plotted with a start arrow as known from automata diagrams
DFG_INPUT_NODE_PATTERNS = [r"^/sensing"]
# RegEx pattern for nodes that shall be marked as system outputs
# These will be plotted with a double border
DFG_OUTPUT_NODE_PATTERNS = [r"^/awapi", r"^/control/external_cmd_converter", "emergency"]
# RegEx for nodes which shall not be plotted in the DFG
DFG_EXCL_NODE_PATTERNS = [r"^/rviz2", r"transform_listener_impl"]

# Whether to compute E2E latencies.
E2E_ENABLED = True
# Whether to plot end-to-end latency information (ignored if E2E_ENABLED is False)
E2E_PLOT = True
# The index of the output message that shall be used in plots that visualize a specific
# message dependency tree. This index has to be 0 <= n < #output messages
E2E_PLOT_TIMESTAMP = 1000
# E2E latency threshold. Every E2E latency higher than this is discarded.
# Set this as low as comfortably possible to speed up calculations.
# WARNING: If you set this too low (i.e. to E2E latencies that plausibly can happen)
# your results will be wrong)
E2E_TIME_LIMIT_S = 2

# All topics containing any of these RegEx patterns are considered output topics in E2E latency calculations
# E.g. r"^/control/" will cover all control topics
E2E_OUTPUT_TOPIC_PATTERNS = [r"emergency/control_cmd"]
# All topics containing any of these RegEx patterns are considered input topics in E2E latency calculations
# E.g. r"^/sensing/" will cover all sensing topics
E2E_INPUT_TOPIC_PATTERNS = [r"^/vehicle/status/", r"^/sensing/(lidar/[^c]|[^l])"]

# E2E paths are uniquely identified by a string like "/topic/1 -> void(Node1)(args1) -> /topic/2 -> void(Node2)(args2) -> void(Node2)(args3) -> ...".
# Certain patterns only occur in initial setup or in scenario switching and can be excluded via RegEx patterns here.
E2E_EXCLUDED_PATH_PATTERNS = [r"NDTScanMatcher", r"^/parameter_events"]


DEBUG = False

# This code overrides the above constants with environment variables, do not edit.
for env_key, env_value in os.environ.items():
    if env_key.startswith("ANA_NB_"):
        key = env_key.removeprefix("ANA_NB_")
        if key not in globals().keys():
            continue
        value = parse_as(type(globals()[key]), env_value)
        globals()[key] = value


# Convert input paths to absolute paths
def _expand_path(path):
    return os.path.realpath(os.path.expandvars(os.path.expanduser(path)))


TRACING_WS_BUILD_PATH = _expand_path(TRACING_WS_BUILD_PATH)
TR_PATH = _expand_path(TR_PATH)
OUT_PATH = _expand_path(OUT_PATH)
BW_PATH = _expand_path(BW_PATH)
CL_PATH = _expand_path(CL_PATH)

os.makedirs(OUT_PATH, exist_ok=True)

print("User Settings:")
for k, v in globals().copy().items():
    if not k.isupper():
        continue
    print(f"  {k:.<40s} := {v}")

In [ ]:
sys.path.append(os.path.join(TRACING_WS_BUILD_PATH, "tracetools_read/"))
sys.path.append(os.path.join(TRACING_WS_BUILD_PATH, "tracetools_analysis/"))
# noinspection PyUnresolvedReferences
from tracetools_read.trace import *
# noinspection PyUnresolvedReferences
from tracetools_analysis.loading import load_file
# noinspection PyUnresolvedReferences
from tracetools_analysis.processor.ros2 import Ros2Handler

from tracing_interop.tr_types import *
from message_tree.message_tree_structure import DepTree
from matching.subscriptions import sanitize

# Load Trace Data

In [ ]:
def _load_traces():
    file = load_file(TR_PATH)
    handler = Ros2Handler.process(file)
    return TrContext(handler)


_tracing_context = cached("tr_objects", _load_traces, [TR_PATH], not CACHING_ENABLED)
_tr_globals = ["nodes", "publishers", "subscriptions", "timers", "timer_node_links", "subscription_objects",
               "callback_objects", "callback_symbols", "publish_instances", "callback_instances", "topics"]

# Help the IDE recognize those identifiers
nodes = publishers = subscriptions = timers = timer_node_links = subscription_objects = callback_objects = callback_symbols = publish_instances = callback_instances = topics = None

for name in _tr_globals:
    globals()[name] = getattr(_tracing_context, name)

print("Done.")

In [ ]:

for topic in sorted(topics, key=lambda t: t.name):
    topic: TrTopic
    print(f"{topic.name:.<120s} | {sum(map(lambda p: len(p.instances), topic.publishers))}")

print("\n[ENTKÄFERN] INPUT TOPICS")
for t in sorted(topics, key=lambda t: t.name):
    for f in E2E_INPUT_TOPIC_PATTERNS:
        if re.search(f, t.name):
            print(f"--[ENTKÄFERN] {f:<30s}:{t.name:.<89s} | {sum(map(lambda p: len(p.instances), t.publishers))}")

print("\n[ENTKÄFERN] OUTPUT TOPICS")
for t in sorted(topics, key=lambda t: t.name):
    for f in E2E_OUTPUT_TOPIC_PATTERNS:
        if re.search(f, t.name):
            print(f"--[ENTKÄFERN] {f:<30s}:{t.name:.<89s} | {sum(map(lambda p: len(p.instances), t.publishers))}")

# Analyze ROS Graph

In [ ]:
import latency_graph.latency_graph_structure as lg


def _make_latency_graph():
    return lg.LatencyGraph(_tracing_context)


lat_graph = cached("lat_graph", _make_latency_graph, [TR_PATH], not CACHING_ENABLED)

## Plot ROS Graph (full)

In [ ]:
%%skip_if_false DFG_ENABLED
%%skip_if_false DFG_PLOT

#################################################
# Plot full DFG, down to the callback level
#################################################

from latency_graph.latency_graph_plots import plot_latency_graph_full

plot_latency_graph_full(lat_graph, _tracing_context, "latency_graph_full")

## Plot Latency Graph (overview)

In [ ]:
%%skip_if_false DFG_ENABLED
%%skip_if_false DFG_PLOT

#################################################
# Plot overview of DFG, down to a certain
# hierarchy level
#################################################

from latency_graph.latency_graph_plots import plot_latency_graph_overview

plot_latency_graph_overview(lat_graph, DFG_EXCL_NODE_PATTERNS, DFG_INPUT_NODE_PATTERNS, DFG_OUTPUT_NODE_PATTERNS, DFG_MAX_HIER_LEVEL, "latency_graph_overview")

# Analyze Message Flow

In [ ]:
%%skip_if_false E2E_ENABLED

from message_tree.message_tree_algorithms import build_dep_trees

end_topics = [t for t in _tracing_context.topics if any(re.search(f, t.name) for f in E2E_OUTPUT_TOPIC_PATTERNS)]

def _build_dep_trees():
    return build_dep_trees(end_topics, lat_graph, _tracing_context, E2E_EXCLUDED_PATH_PATTERNS, E2E_TIME_LIMIT_S)

try:
    trees = cached("trees", _build_dep_trees, [TR_PATH], not CACHING_ENABLED)
except Exception as e:
    import traceback
    print(e)
    traceback.print_exc()

In [ ]:
%%skip_if_false E2E_ENABLED
%%skip_if_false BW_ENABLED


def parse_bytes(string):
    match string[-1]:
        case 'K':
            exponent = 1e3
        case 'M':
            exponent = 1e6
        case _:
            exponent = 1

    num = float(string.split(" ")[0])
    return num * exponent


def bytes_str(bytes):
    if bytes >= 1024 ** 2:
        return f"{bytes / (1024 ** 2):.2f} MiB"
    if bytes >= 1024:
        return f"{bytes / 1024:.2f} KiB"
    return f"{bytes:.0f} B"


bw_files = glob.glob(os.path.join(BW_PATH, "*.log"))
msg_sizes = {}
for bw_file in bw_files:
    with open(bw_file) as f:
        lines = f.readlines()
        topic = os.path.splitext(os.path.split(bw_file)[1])[0].replace("__", "/")

        if not lines or re.match(f"^\s*$", lines[-1]):
            #print(f"No data for {topic}")
            continue

        line_pattern = re.compile(
            r"(?P<bw>[0-9.]+ [KM]?)B/s from (?P<n_msgs>[0-9.]+) messages --- Message size mean: (?P<mean>[0-9.]+ [KM]?)B min: (?P<min>[0-9.]+ [KM]?)B max: (?P<max>[0-9.]+ [KM]?)B\n")
        m = re.fullmatch(line_pattern, lines[-1])
        if m is None:
            print(f"Line could not be parsed in {topic}: '{lines[-1]}'")
            continue

        msg_sizes[topic] = {'bw': parse_bytes(m.group("bw")),
                            'min': parse_bytes(m.group("min")),
                            'mean': parse_bytes(m.group("mean")),
                            'max': parse_bytes(m.group("max"))}

In [ ]:
%%skip_if_false E2E_ENABLED
%%skip_if_false E2E_PLOT
%%skip_if_false DEBUG


fig, ax = plt.subplots(num="e2e_plot")
DS = 1
times = [tree.head.timestamp - trees[0].head.timestamp for tree in trees]

ax2 = ax.twinx()
ax2.plot(times, list(map(lambda paths: sum(map(len, paths)) / len(paths), e2e_pathss)), color="orange")
ax2.fill_between(times,
                 list(map(lambda paths: min(map(len, paths)), e2e_pathss)),
                 list(map(lambda paths: max(map(len, paths)), e2e_pathss)),
                 alpha=.3, color="orange")

ax.plot(times, [np.mean(e2es) for e2es in e2ess])
ax.fill_between(times, [np.min(e2es) for e2es in e2ess], [np.max(e2es) for e2es in e2ess], alpha=.3)


def scatter_topic(topic_name, y=0, **scatter_kwargs):
    for pub in topics.by_name[topic_name].publishers:
        if not pub:
            continue

        inst_timestamps = [inst.timestamp - trees[0].head.timestamp for inst in pub.instances if
                           inst.timestamp >= trees[0].head.timestamp]
        scatter_kwargs_default = {"marker": "x", "color": "indianred"}
        scatter_kwargs_default.update(scatter_kwargs)
        ax.scatter(inst_timestamps, np.full(len(inst_timestamps), fill_value=y), **scatter_kwargs_default)


scatter_topic("/autoware/engage")
scatter_topic("/planning/scenario_planning/parking/trajectory", y=-.04, color="cadetblue")
scatter_topic("/planning/scenario_planning/lane_driving/trajectory", y=-.08, color="darkgreen")

ax.set_xlabel("Simulation time [s]")
ax.set_ylabel("End-to-End latency [s]")
ax2.set_ylabel("End-to-End path length")
None

In [ ]:
%%skip_if_false DEBUG

import pickle
with open("trees.pkl", "rb") as f:
    trees = pickle.load(f)

In [ ]:
%%skip_if_false E2E_ENABLED

from message_tree.message_tree_algorithms import e2e_paths_sorted_desc
from message_tree.message_tree_plots import e2e_breakdown_type_hist

trees_paths = [e2e_paths_sorted_desc(tree, E2E_INPUT_TOPIC_PATTERNS) for tree in tqdm(trees, mininterval=10.0,
                                                                                      desc="Extracting E2E paths")]
all_paths = [p for paths in trees_paths for p in paths]
#all_e2e_items = [i for p in all_paths for i in p]

In [ ]:
%%skip_if_false E2E_ENABLED
%%skip_if_false E2E_PLOT

from message_tree.message_tree_algorithms import e2e_latency_breakdown

conv_items = [i for p in tqdm(all_paths, mininterval=5.0, desc="Calculating E2E latency breakdowns")
              for i in e2e_latency_breakdown(p)]
e2e_breakdown_type_hist(conv_items)

None


In [ ]:
%%skip_if_false E2E_ENABLED

from message_tree.message_tree_algorithms import aggregate_e2e_paths

cohorts = aggregate_e2e_paths(all_paths)
cohort_pairs = [(k, v) for k, v in cohorts.items()]
cohort_pairs.sort(key=lambda kv: len(kv[1]), reverse=True)

path_records = [{"path": path_key,
                 "timestamp": path[-1].timestamp,
                 "e2e_latency": path[-1].timestamp - path[0].timestamp} \
                for path_key, paths in cohort_pairs for path in paths if path]

out_df = pd.DataFrame.from_records(path_records)
out_df.to_csv(os.path.join(OUT_PATH, "e2e.csv"), sep="\t", index=False)

mode_cohort_key, mode_cohort = cohort_pairs[0]

In [ ]:
%%skip_if_false E2E_ENABLED
%%skip_if_false E2E_PLOT

from message_tree.message_tree_plots import e2e_breakdown_inst_stack


mode_cohort_breakdown = [e2e_latency_breakdown(p) for p in mode_cohort[:200]]
print(len(mode_cohort))
print(mode_cohort_key.replace(" -> ", "\n -> "))
e2e_breakdown_inst_stack(*mode_cohort_breakdown)

In [ ]:
%%skip_if_false E2E_ENABLED
%%skip_if_false DEBUG

import pickle

with open("trees.pkl", "wb") as f:
    pickle.dump(trees, f)